In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import random
import cv2
import pickle
from pprint import pprint
from PIL import Image
from IPython.display import display

from utils import *
from image_transform import *
from loss_function import *
from model import *

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from torchvision import transforms
from ranger.ranger2020 import Ranger
from pytorchcv.models import efficientnet

plt.style.use("seaborn")


In [7]:
# 檔案設定:
root = Root(r'C:\AI\Selected_Topics_in_Visual_Recognition_using_Deep_Learning\Final')
trainDir = root('train_images')
testDir  = root('test_images' )
extraDir = root('extra_images')
modelDir = root('models')

# 資料集設定:
split = 0.8
numClasses = 5

# 訓練設定:
vaild     = True
device    = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
imgShape  = [300, 300]
batchSize = 16
imgMean   = [0.485, 0.456, 0.406]
imgStd    = [0.229, 0.224, 0.225]
lr        = 1e-5

# 測試設定:
threshold = [0.5, 1.5, 2.5, 3.5]


In [8]:
trainDF, validDF = pd.read_csv(root('train.csv'))[['id_code', 'diagnosis']], pd.read_csv(root('valid.csv'))[['id_code', 'diagnosis']]
extraDF = BuildImagePathsDataFrame(extraDir)
allDF = pd.concat([trainDF, validDF], ignore_index=True)

trainTransforms = transforms.Compose([
    CropImageFromGray(),
    ClearImage(),
    transforms.ToPILImage(),
    transforms.Resize(imgShape),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomAffine(180, shear=0.2, resample=Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(imgMean, imgStd)
])

validTransforms = transforms.Compose([
    CropImageFromGray(),
    ClearImage(),
    transforms.ToPILImage(),
    transforms.Resize(imgShape),
    transforms.ToTensor(),
    transforms.Normalize(imgMean, imgStd)
])

trainDS = ImageDataset(trainDF, True , 'id_code', 'diagnosis', transform=trainTransforms, imageDir=trainDir)
validDS = ImageDataset(validDF, True , 'id_code', 'diagnosis', transform=validTransforms, imageDir=trainDir)
extraDS = ImageDataset(extraDF, False, 'id_code', 'diagnosis', transform=validTransforms, imageDir=extraDir)

trainDL = DataLoader(trainDS, batch_size=batchSize, shuffle=True , num_workers=12)
validDL = DataLoader(validDS, batch_size=batchSize, shuffle=False, num_workers=12)
extraDL = DataLoader(extraDS, batch_size=batchSize, shuffle=True , num_workers=12)


In [9]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)  

    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'
    
    @staticmethod
    def gem(x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)


class Detector(nn.Module):
    def __init__(self, numClasses, threshold):
        super(Detector, self).__init__()
        self.numClasses = numClasses
        self.threshold = torch.tensor([(2 * t - (numClasses - 1)) / (numClasses - 1) for t in threshold])

        net = efficientnet.efficientnet_b2c(pretrained=True)
        net.features.final_pool = GeM()
        inputDim = net.output.fc.in_features
        self.cnn = net.features
        self.flat = nn.Flatten()
        self.drop = nn.Dropout(p=0.2)
        self.cls = nn.Sequential(
            nn.Linear(inputDim, numClasses, bias=False)
        )
        self.reg = nn.Sequential(
            nn.Linear(inputDim, 1, bias=False)
        )

        del net
    

    def forward(self, x):
        h = self.drop(self.flat(self.cnn(x)))
        c = self.cls(h)
        r = self.reg(h)
        return c, r
    

    def ConvertRegressionToClass(self, r):
        r = r.detach().to(self.threshold.device)
        return torch.sum(r > self.threshold, dim=1)



In [10]:
# Model: ---------------------------------------------------------------------
model = Detector(numClasses, threshold)
model.requires_grad_()
model = model.to(device)

# Loss functions: -----------------------------------------------------------
criterion1 = nn.CrossEntropyLoss().to(device)
criterion2 = nn.SmoothL1Loss().to(device)

# Optimizer: -----------------------------------------------------------------
optimizer = optim.AdamW(model.parameters(), weight_decay=1e-4, lr=lr)
scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=100, T_mult=3, eta_min=1e-6)

# Load state: ---------------------------------------------------------------
# _, model, optimizer, scheduler = LoadState(model, 
#                                            optimizer, 
#                                            scheduler, 
#                                            os.path.join(modelDir, 'state10_epoch50_trainRegAcc=0.8_trainClsAcc=0.9138.pth'))


In [6]:
# Validation process:
def Validate(model, validDL, device):
    clsAcc = 0.
    regAcc = 0.
    num = 0
    model.eval()
    with torch.no_grad():
        for imgs, labels in validDL:
            imgs = imgs.to(device)
            labels = labels.to(device)

            c, r = model(imgs)
            clsAcc += GetAccuracy(c, labels).item()
            regAcc += GetAccuracy(model.ConvertRegressionToClass(r), labels).item()
            num += 1

        clsAcc /= num
        regAcc /= num
    
    return clsAcc, regAcc


# Training process:
save = True
startEpoch = 1
endEpoch   = 50
numBatch = len(trainDS) // batchSize + 1
maxValidAcc = 0.
for epoch in range(startEpoch, endEpoch + 1):
    trainClsAccuracy = 0.
    trainRegAccuracy = 0.
    for i, (imgs, labels) in enumerate(trainDL):
        # Inputs & targets:
        imgs = imgs.to(device)
        labels = labels.to(device)
        target = ((2 * labels - (numClasses - 1)) / (numClasses - 1)).view(-1, 1).to(device).detach()

        # Train model:
        model.train()
        optimizer.zero_grad()
            
        c, r = model(imgs)
        loss = 0.25 * criterion1(c, labels) + criterion2(r, target)

        loss.backward()
        optimizer.step()
        scheduler.step()
        trainClsAccuracy += GetAccuracy(c, labels).item()
        trainRegAccuracy += GetAccuracy(model.ConvertRegressionToClass(r), labels).item()
        sys.stdout.write(f"\r|Epoch {epoch}/{endEpoch}|Batch {i + 1}/{numBatch}| => Train Classifier Acc = {round(trainClsAccuracy / (i + 1), 4)}, Train Regressor Acc = {round(trainRegAccuracy / (i + 1), 4)}")

    # Test model:
    if epoch % 1 == 0 and vaild:
        validClsAccuracy, validRegAccuracy = Validate(model, validDL, device)
        print(f", Valid Classifier Acc = {round(validClsAccuracy, 4)}, Valid Regressor Acc = {round(validRegAccuracy, 4)}")
    else:
        print(" ")
    
    if save:
        name = f'state11_epoch{epoch}_trainRegAcc={round(trainRegAccuracy / numBatch, 4)}_trainClsAcc={round(trainClsAccuracy / numBatch, 4)}.pth'
        path = os.path.join(modelDir, name)
        SaveState(model, optimizer, scheduler, epoch, path)


|Epoch 51/60|Batch 206/206| => Train Classifier Acc = 0.1029, Train Regressor Acc = 0.2588, Valid Classifier Acc = 0.1141, Valid Regressor Acc = 0.2125
|Epoch 52/60|Batch 206/206| => Train Classifier Acc = 0.1193, Train Regressor Acc = 0.2631, Valid Classifier Acc = 0.1359, Valid Regressor Acc = 0.2207
|Epoch 53/60|Batch 206/206| => Train Classifier Acc = 0.1712, Train Regressor Acc = 0.2722, Valid Classifier Acc = 0.2917, Valid Regressor Acc = 0.2589
|Epoch 54/60|Batch 206/206| => Train Classifier Acc = 0.3157, Train Regressor Acc = 0.3195, Valid Classifier Acc = 0.4221, Valid Regressor Acc = 0.235
|Epoch 55/60|Batch 206/206| => Train Classifier Acc = 0.4358, Train Regressor Acc = 0.3523, Valid Classifier Acc = 0.5341, Valid Regressor Acc = 0.3486
|Epoch 56/60|Batch 206/206| => Train Classifier Acc = 0.4764, Train Regressor Acc = 0.3669, Valid Classifier Acc = 0.5422, Valid Regressor Acc = 0.3759
|Epoch 57/60|Batch 206/206| => Train Classifier Acc = 0.5269, Train Regressor Acc = 0.394

KeyboardInterrupt: 